In [58]:
import pandas as pd

### Disease -> CUI

In [2]:
broad_dict_raw = pd.read_csv("../mapping data/broad_new_full_dict.txt", sep="|", header=None)

# names columns for extracting
broad_dict_raw.columns = ["name","1",'2',"cui",'4','5']

/Users/neneko/Documents/GitHub/PubHealthData/drug-disease/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
broad_dict_raw[:10]

,name,1,2,cui,4,5
0,Dipalmitoyl Phosphatidylcholine,2.0,2,C0000039,Pharmacologic Substance,CHEM
1,Dipalmitoylglycerophosphocholine,2.0,2,C0000039,Pharmacologic Substance,CHEM
2,Dipalmitoyllecithin,2.0,2,C0000039,Pharmacologic Substance,CHEM
3,Branching Enzyme,3.0,3,C0000052,"Amino Acid, Peptide, or Protein",CHEM
4,Branching Glycosyltransferase,3.0,3,C0000052,"Amino Acid, Peptide, or Protein",CHEM
5,Starch Branching Enzyme,3.0,3,C0000052,"Amino Acid, Peptide, or Protein",CHEM
6,1 Alkyl 2 Acylphosphatidates,4.0,4,C0000074,Lipid,CHEM
7,1-Alkyl-2-Acylphosphatidates,4.0,4,C0000074,Lipid,CHEM
8,1 Carboxyglutamic Acid,5.0,5,C0000084,Biologically Active Substance,CHEM
9,1-Carboxyglutamic Acid,5.0,5,C0000084,Biologically Active Substance,CHEM


In [4]:
broad_dict_raw["name"][9000:9100]

9000             Suffocations
9001                 asphyxia
9002          Suffocation NOS
9003             asphyxiation
9004              suffocating
                ...          
9095    Association Learnings
9096              Asta C 4898
9097           Astasia Abasia
9098        Hysterical ataxia
9099           astasia-abasia
Name: name, Length: 100, dtype: object

In [5]:
# create string - CUI pairs
disease_name = broad_dict_raw["name"].apply(lambda x: x.lower().replace(",","").replace(" / ", "/") if type(x) == str else "")
cui = broad_dict_raw["cui"].apply(lambda x: x.replace(";", "|"))
str_cui_dic = dict(zip(disease_name,cui))

In [6]:
disease_name[:10]

0     dipalmitoyl phosphatidylcholine
1    dipalmitoylglycerophosphocholine
2                 dipalmitoyllecithin
3                    branching enzyme
4       branching glycosyltransferase
5             starch branching enzyme
6        1 alkyl 2 acylphosphatidates
7        1-alkyl-2-acylphosphatidates
8              1 carboxyglutamic acid
9              1-carboxyglutamic acid
Name: name, dtype: object

In [7]:
cui[:10]
# multiple strings might map to the same CUI

0    C0000039
1    C0000039
2    C0000039
3    C0000052
4    C0000052
5    C0000052
6    C0000074
7    C0000074
8    C0000084
9    C0000084
Name: cui, dtype: object

In [8]:
import numpy as np
df = pd.read_csv("output/drug_disease.csv").replace(np.nan, '', regex=True)
df_rx = df[df["RX/OTC"] != 'otc']


In [9]:
def _clean_generic_name(x):

    x = x.replace(" )","")
    x = x.replace("|and ","/")

    return x


In [10]:
generic_names = df_rx["generic name"].apply(lambda x: _clean_generic_name(x))
brand_names = df_rx["brand names"]

In [11]:
generic_names[:10]

0             vasopressin
1             vasopressin
2             bethanechol
3             bethanechol
4             bethanechol
7             vasopressin
8     medroxyprogesterone
9                        
10    medroxyprogesterone
11          norethindrone
Name: generic name, dtype: object

In [12]:
brand_names[:10]

0                                  vasostrict|pitressin
1                                            vasostrict
2                                     duvoid|urecholine
3                                     duvoid|urecholine
4                                     duvoid|urecholine
7                                  vasostrict|pitressin
8                                                      
9                                                      
10                                              provera
11    aygestin|camila|errin|jolivette|nora-be|ortho ...
Name: brand names, dtype: object

In [13]:
def _unique(lst):
    '''returns a list with unique values'''
    return list(np.unique(np.array(lst)))

In [14]:
generic_names_unique = generic_names
generic_names_unique = _unique(generic_names)
generic_names_unique.remove("")
len(generic_names_unique)

1994

In [15]:
generic_missing_terms_dict = {}

In [16]:
mapped_num = 0
# map our data
cui_codes_list_gen = []
prev_gen_name_set = []

# keep track of what is not found
missed_generic_names = []
i = 0
for name in generic_names:

    cur_cuis = []
    if name in generic_missing_terms_dict:
        cur_cuis.append(generic_missing_terms_dict[name])

    # name = name.replace(" prophylaxis", "")
    # name = name.replace(" human","").replace("human", "")

    if name in str_cui_dic:
         cur_cuis.append(str_cui_dic[name])

    cur_cuis = _unique(cur_cuis)

    if name not in prev_gen_name_set:
        if len(cur_cuis) > 0:
            mapped_num += 1
        else:
            missed_generic_names.append(name)
        prev_gen_name_set.append(name)

    cui_codes_list_gen.append("|".join(cur_cuis))
    i += 1

In [17]:
mapping_rate_str2rxn_generic = mapped_num/len(generic_names_unique)
mapping_rate_str2rxn_generic

0.7286860581745236

In [18]:
df_rx["cui_from_generic"] = cui_codes_list_gen
#df_rx.to_csv("output/PheCode_rxNorm.csv")

/Users/neneko/Documents/GitHub/PubHealthData/drug-disease/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
len(missed_generic_names)

542

In [20]:
missed_generic_names_df = pd.DataFrame(missed_generic_names)
missed_generic_names_df.columns = ["generic names"]
missed_generic_names_df["CUIs"] = ""*len(missed_generic_names)

missed_generic_names_df.to_csv("missed mappings/missed CUIs from generic name.csv", index=False)

In [21]:
str_cui_dic["venetoclax"]

KeyError: 'venetoclax'

In [23]:
str_cui_dic["gilteritinib"]

KeyError: 'gilteritinib'

In [24]:
str_cui_dic["elbasvir / grazoprevir"]

KeyError: 'elbasvir / grazoprevir'

In [25]:
missed_generic_names_df.sample(10)

,generic names,CUIs
194,ivacaftor/tezacaftor,
496,crizanlizumab,
386,aspirin|carisoprodol/codeine,
131,plecanatide,
16,gilteritinib,
124,ledipasvir/sofosbuvir,
0,levonorgestrel emergency contraceptive,
404,cenegermin,
116,empagliflozin,
515,avatrombopag,


### Map brand names to CUI

In [26]:
brand_names_unique = brand_names
brand_names_unique = _unique(brand_names)
brand_names_unique.remove("")
len(brand_names_unique)

1994

In [27]:
brand_missing_terms_dict = {}

In [28]:
mapped_num = 0
# map our data
cui_codes_list_brand = []

prev_brand_name_set = []

# keep track of what is not found
missed_brand_names = []
i = 0
for name in brand_names:

    cur_cuis = []
    name = name.replace(" )","")
    if name in brand_missing_terms_dict:
        cur_cuis.append(brand_missing_terms_dict[name])

    # name = name.replace(" prophylaxis", "")
    # name = name.replace(" human","").replace("human", "")

    if name in str_cui_dic:
         cur_cuis.append(str_cui_dic[name])

    cur_cuis = _unique(cur_cuis)

    if name not in prev_brand_name_set:
        if len(cur_cuis) > 0:
            mapped_num += 1
        else:
            missed_brand_names.append(name)
        prev_brand_name_set.append(name)

    cui_codes_list_brand.append("|".join(cur_cuis))
    i += 1

In [29]:
mapping_rate_str2rxn_brand = mapped_num/len(brand_names_unique)
mapping_rate_str2rxn_brand

0.35707121364092276

In [30]:
df_rx["cui_from_brand"] = cui_codes_list_brand
#df_rx.to_csv("output/PheCode_rxNorm.csv")


/Users/neneko/Documents/GitHub/PubHealthData/drug-disease/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
len(missed_brand_names)

1283

In [32]:
missed_brand_names_df = pd.DataFrame(missed_brand_names)
missed_brand_names_df.columns = ["brand names"]
missed_brand_names_df["CUIs"] = ""*len(missed_brand_names)

missed_brand_names_df.to_csv("missed mapping/missed CUIs from brand name.csv", index=False)


In [33]:
str_cui_dic["venetoclax"]

KeyError: 'venetoclax'

In [34]:
str_cui_dic["gilteritinib"]

KeyError: 'gilteritinib'

In [35]:
str_cui_dic["elbasvir / grazoprevir"]

KeyError: 'elbasvir / grazoprevir'

In [36]:
missed_brand_names_df.sample(10)

,brand names,CUIs
1167,abilify maintena|aristada,
1075,stalevo 100|stalevo 125|stalevo 150|stalevo 20...,
994,metaxall|skelaxin,
415,bosulif,
737,aliqopa,
527,8-mop|oxsoralen-ultra,
1051,symproic,
702,ak-con|albalon|allersol|clear eyes|clear eyesr...,
688,tazverik,
648,lescol|lescol xl,


In [37]:
len(generic_names_unique)


1994

In [38]:
len(brand_names_unique)

1994

CUI - RXNorms
Using the broad_new_dictionary file, link rxnorms(string) to CUI

In [39]:
rx_norms_df = pd.read_csv("../mapping data/RXNORM-ingredient-base.csv")

In [40]:
rx_norms_df.columns

Index(['base', 'base_str', 'ingredient', 'ingredient_str'], dtype='object')

In [41]:
rx_norm_strings = rx_norms_df["base_str"].apply(lambda x: x.replace(" / ", "/")).to_list()
rx_norm_strings += rx_norms_df["ingredient_str"].apply(lambda x: x.replace(" / ", "/")).to_list()

In [42]:
len(rx_norm_strings)

188890

In [43]:
mapped_cuis = []
missed_strings = []
for string in rx_norm_strings:
    if string in str_cui_dic:
        mapped_cuis.append(str_cui_dic[string])
    else:
        missed_strings.append(string)

In [44]:
# Mappinge rate:
string_rxnorm_mapping_rate = len(mapped_cuis)/len(rx_norm_strings)
string_rxnorm_mapping_rate

0.5779977764836678

In [45]:
cui_rxnorm_dic = dict(zip(mapped_cuis, rx_norm_strings))

In [63]:
# Map the CUIs we have to rx norms

# first join gen-cui list and brand-cui list

combined_cui = []
for i in range(len(cui_codes_list_gen)):
    cur_cui = ""
    if cui_codes_list_gen[i]:
        cur_cui += cui_codes_list_gen[i]
        if cui_codes_list_brand[i]:
            cur_cui = cur_cui + "|" + cui_codes_list_brand[i]
    elif not cui_codes_list_gen[i]:
        if cui_codes_list_brand[i]:
            cur_cui = cui_codes_list_brand[i]

    combined_cui.append(cur_cui)
    i += 1

In [64]:
pd.DataFrame(combined_cui).sample(50)

,0
8281,C0011777
1023,C0034414|C0337453
11084,C0286651|C1272460|C1705112
12325,C2725260|C1272460|C1705112
10802,C1695579|C1272460|C1705112
10072,C0024002|C1272460|C1705112
6860,C0010583
11429,C0360584
13717,
9205,


In [65]:
# store this combined cui to the output file
df_rx["cui_combined_gen_brand"] = combined_cui

/Users/neneko/Documents/GitHub/PubHealthData/drug-disease/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
df_rx.to_csv("output/PheCode_rxNorm.csv")

In [67]:
non_empty_cuis_df = pd.DataFrame(combined_cui).replace("", np.nan, regex=True).dropna()


In [69]:
mapping_rate_str_cui = len(non_empty_cuis_df) / len(combined_cui)
mapping_rate_str_cui

0.9204048140043763

In [74]:
# map combined cui to rxnorms using the cui_rxnorm_dic we just created

cui_rxnorms = []
missed_cuis = []
mapped_num_cui_rxnorm = 0

for row in combined_cui:
    cur_rxnorm = []
    for c in row.split("|"):
        if c in cui_rxnorm_dic:
            cur_rxnorm.append(cui_rxnorm_dic[c])

        else:
            # only track which cui is missing from the dict we are using
            missed_cuis.append(c)

        cur_rxnorm = _unique(cur_rxnorm)

    if cur_rxnorm:
        mapped_num_cui_rxnorm += 1

    cui_rxnorms.append("|".join(cur_rxnorm))

    missed_cuis = _unique(missed_cuis)

In [53]:
cui_rxnorm_mapping_rate = mapped_num_cui_rxnorm / len(combined_cui)
cui_rxnorm_mapping_rate

0.6246580962800875

In [71]:
net_mapping_rate = mapping_rate_str_cui * cui_rxnorm_mapping_rate
net_mapping_rate

0.5749383189230017

In [75]:
len(missed_cuis)

1165

In [80]:
missed_terms_df = pd.DataFrame(missed_cuis)
missed_terms_df.columns = ["CUIs"]
missed_terms_df.to_csv("missed output/missed CUIs from combined names.csv", index=False)

In [77]:
df_rx["rxnorms_from_CUI"] = cui_rxnorms
df_rx.to_csv("PheCode_rxNorm.csv")

/Users/neneko/Documents/GitHub/PubHealthData/drug-disease/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
